# 사용할 데이터베이스 SQL 로그

create table TeamVSTeam (`경기 일자` timestamp, `플레이한 팀1 이름` varchar (20), `플레이한 팀2 이름` varchar(20), `플레이한 팀1 점수` int, `플레이한 팀2 점수` int, `경기 기록 링크` text, `중계한 TV 채널` varchar(100), `중계한 라디오 채널` varchar(100), `플레이 구장` varchar(25), `비고` varchar(200), PRIMARY KEY(`경기 일자`, `플레이한 팀1 이름`, `플레이한 팀2 이름`));

alter table TeamVSTeam MODIFY `플레이한 팀1 점수` int DEFAULT -1;

alter table TeamVSTeam MODIFY `플레이한 팀2 점수` int DEFAULT -1;

alter table TeamVSTeam MODIFY `경기 기록 링크` varchar(1000) DEFAULT "N/A";

alter table TeamVSTeam MODIFY `중계한 TV 채널` varchar(200) DEFAULT 'N/A';

alter table TeamVSTeam MODIFY `중계한 라디오 채널` varchar(200) DEFAULT 'N/A';

alter table TeamVSTeam MODIFY `플레이 구장` varchar(25) DEFAULT 'N/A';

alter table TeamVSTeam MODIFY `비고` varchar(50) DEFAULT 'N/A';

# 사용할 모듈을 임포트

In [ ]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import time
import random
#from __future__ import print_function
import pymysql

# 크롬 드라이버를 이용한 웹 크롤링 준비

In [ ]:
webCrawl = webdriver.Chrome("/Users/stories2/Python/chromedriver")

# 사용할 selenium 함수를 재정의

In [ ]:
def ConnectToTargetUrl(url):
    webCrawl.get(url)

In [ ]:
def GetSpecialTagData(tagName, webCrawl):
    return webCrawl.find_element_by_tag_name(tagName)

In [ ]:
def GetSpecialIdData(idName, webCrawl):
    return webCrawl.find_element_by_id(idName)

In [ ]:
def GetSpecialClassData(className, webCrawl):
    return webCrawl.find_element_by_class_name(className)

In [ ]:
def GetSpecialXPathData(XPath, webCrawl):
    return webCrawl.find_element_by_xpath(XPath)

In [ ]:
def GetSpecialTagDatas(tagName, webCrawl):
    return webCrawl.find_elements_by_tag_name(tagName)

In [ ]:
def GetSpecialIdDatas(idName, webCrawl):
    return webCrawl.find_elements_by_id(idName)

In [ ]:
def GetSpecialClassDatas(className, webCrawl):
    return webCrawl.find_elements_by_class_name(className)

In [ ]:
def GetSpecialXPathDatas(XPath, webCrawl):
    return webCrawl.find_elements_by_xpath(XPath)

# 데이터베이스 연동 부분

In [ ]:
def GetDBCursor(connectorOfDB):
    return connectorOfDB.cursor()

In [ ]:
def ExecuteQuery(cursorOfDB, query):
    cursorOfDB.execute(query)

In [ ]:
def InsertDataToDB(query):
    connection = pymysql.connect(host = "lamb.kangnam.ac.kr", user = "stories2", password = "toortoor%^%", db = "stories2", charset = "utf8")
    #print(query)
    ExecuteQuery(GetDBCursor(connection), query)
    connection.commit()
    connection.close()

In [ ]:
def MakeQueryWithParameter(dateTime, teams, score, link, tv, radio, place, etc):
    return "insert into TeamVSTeam value('" + dateTime + "', " + teams + ", " + score + ", " + link + ", " + tv + ", " + radio + ", " + place + ", " + etc + ");"

# 실제 웹 데이터에서 필요한 특정 정보를 가져오는 부분

전 년/월 경기를 로드할 버튼

In [ ]:
def GetPreviewsBaseballGameData(webCrawl):
    return GetSpecialIdData("cphContainer_cphContents_btnPreDate", webCrawl);

로드하는 해당 버튼을 클릭

In [ ]:
def GetPreviewsBaseballGameDataClick(webCrawl):
    GetPreviewsBaseballGameData(webCrawl).click();
    time.sleep(0.5)

로드된 년/월에 대한 정보를 가져옴

디폴트 리턴 값: 0000-00

업데이트된 리턴 값: 2016-10

In [ ]:
def GetNowSearchedDate(webCrawl):
    selectedYear = "0000"
    selectedMonth = "00"
    
    webCrawlGetSelectYearOptions = GetSpecialIdData("cphContainer_cphContents_ddlYear", webCrawl)
    webCrawlGetSelectMonthOptions = GetSpecialIdData("cphContainer_cphContents_ddlMonth", webCrawl)
    
    selectedYear = GetSpecialXPathData("./option[@selected='selected']", webCrawlGetSelectYearOptions).text.encode('utf-8')
    selectedMonth = GetSpecialXPathData("./option[@selected='selected']", webCrawlGetSelectMonthOptions).text.encode('utf-8')
    
    selectedYear = selectedYear.replace("년", "")
    selectedMonth = selectedMonth.replace("월", "")
    return selectedYear + "-" + selectedMonth

로드된 해당 년/월에 대한 경기 이력들 가져오기

여러개의 tr태그들이 리턴됨

In [ ]:
def GetSearchedScheduleBoardList(webCrawl):
    webCrawlScheduleBoard = GetSpecialClassData("scheduleBoard", webCrawl)
    webCrawlScheduleBoardBody = GetSpecialTagData("tbody", webCrawlScheduleBoard)
    return GetSpecialTagDatas("tr", webCrawlScheduleBoardBody)

## 각 tr태그 마다 적용할 내용들

tr 태그 밑에 속한 td 태그들을 크롤링함

여러개의 td태그들이 리턴됨

In [ ]:
def GetSearchedScheduleTagOfTD(webCrawl):
    return GetSpecialTagDatas("td", webCrawl)

경기 시작 시간을 조회

기본 값: 00:00

업데이트된 값: 17:00

In [ ]:
def GetBaseballPlayedTime(webCrawl):
    baseballGamePlayedTime = "00:00"
    try:
        #print GetSpecialClassData("time", webCrawl)
        #print GetSpecialXPathData("./td[@class='time']", webCrawl)
        #print "debug " + str(baseballGamePlayedTime)
        #print webCrawl.find_element_b("time").text
        baseballGamePlayedTime = GetSpecialClassData("time", webCrawl).text
    except:
        baseballGamePlayedTime = "00:00"
    return baseballGamePlayedTime

날짜를 조회

기본 값: 00

업데이트된 값: 01

In [ ]:
def GetBaseballPlayedDate(webCrawl):
    baseballGamePlayedDate = "00"
    try:
        webCrawlBaseballPlayedDateClass = GetSpecialClassData("day", webCrawl)
        baseballGamePlayedDate = GetSpecialTagData("span", webCrawlBaseballPlayedDateClass).text
        baseballGamePlayedDate = baseballGamePlayedDate[3:5]
    except:
        baseballGamePlayedDate = "00"
    return baseballGamePlayedDate

플래이한 팀의 이름을 가져옴

기본 값: 'N/A', 'N/A'

업데이트된 값: 'SK', 'LG'

In [ ]:
def GetBaseballPlayedTeamsName(webCrawl):
    baseballGamePlayedTeamsName = "'N/A', 'N/A'"
    try:
        webCrawlBaseballPlayedTeamsClass = GetSpecialClassData("play", webCrawl)
        webCrawlBaseballPlayedTeamsTags = GetSpecialTagDatas("span", webCrawlBaseballPlayedTeamsClass)
        baseballGamePlayedTeamsName = "'" + webCrawlBaseballPlayedTeamsTags[0].text + "', '" + webCrawlBaseballPlayedTeamsTags[len(webCrawlBaseballPlayedTeamsTags) - 1].text + "'"
    except:
        baseballGamePlayedTeamsName = "'N/A', 'N/A'"
    return baseballGamePlayedTeamsName

플래이한 경기 결과를 가져옴

기본 값: -1, -1

업데이트된 값: 5, 0

In [ ]:
def GetBaseballPlayedScore(webCrawl):
    baseballGamePlayedScore = "-1, -1"
    try:
        webCrawlBaseballPlayedScoreClass = GetSpecialClassData("play", webCrawl)
        webCrawlBaseballPlayedScoreTags = GetSpecialTagDatas("span", webCrawlBaseballPlayedScoreClass)
        if len(webCrawlBaseballPlayedScoreTags) < 5:
            return baseballGamePlayedScore
        baseballGamePlayedScore = webCrawlBaseballPlayedScoreTags[1].text + ", "+ webCrawlBaseballPlayedScoreTags[3].text
    except:
        baseballGamePlayedScore = "-1, -1"
    return baseballGamePlayedScore

경기 세부 기록 링크

기본 값: 'N/A'

업데이트된 값: 'http://~~~'

In [ ]:
def GetBaseballGameDetailLink(webCrawl):
    baseballGameDetailLink = "'N/A'"
    try:
        webCrawlBaseballGameDetailLinkClass = GetSpecialClassData("relay", webCrawl)
        webCrawlBaseballGameDetailLinkTag = GetSpecialTagData("a", webCrawlBaseballGameDetailLinkClass)
        baseballGameDetailLink = "'" + webCrawlBaseballGameDetailLinkTag.get_attribute("href") + "'"
    except:
        baseballGameDetailLink = "'N/A'"
    return baseballGameDetailLink

중계한 TV 채널

기본 값: 'N/A'

업데이트된 값: 'MS-T SKY-T'

In [ ]:
def GetBaseballBroadcastTV(webCrawl):
    baseballGameBroadcastTV = "'N/A'"
    try:
        webCrawlBaseballGameBroadcastTVClass = GetSpecialClassData("tv", webCrawl)
        baseballGameBroadcastTV = webCrawlBaseballGameBroadcastTVClass.text
        baseballGameBroadcastTV.replace("\"", "")
        baseballGameBroadcastTV.replace("<br>", " ")
        if baseballGameBroadcastTV == "" or baseballGameBroadcastTV == " ":
            baseballGameBroadcastTV = "'N/A'"
        else:
            baseballGameBroadcastTV = "'" + baseballGameBroadcastTV + "'"
    except:
        baseballGameBroadcastTV = "'N/A'"
    return baseballGameBroadcastTV

중계한 라디오 채널

기본 값: 'N/A'

업데이트된 값: 'KNN-R PM-R'

In [ ]:
def GetBaseballBroadcastRadio(webCrawl):
    baseballGameBroadcastRadio = "'N/A'"
    try:
        webCrawlBaseballGameBroadcastRadioClass = GetSpecialClassData("radio", webCrawl)
        baseballGameBroadcastRadio = webCrawlBaseballGameBroadcastRadioClass.text
        baseballGameBroadcastRadio.replace("\"", "")
        baseballGameBroadcastRadio.replace("<br>", " ")
        if baseballGameBroadcastRadio == "" or baseballGameBroadcastRadio == " ":
            baseballGameBroadcastRadio = "'N/A'"
        else:
            baseballGameBroadcastRadio = "'" + baseballGameBroadcastRadio + "'"
    except:
        baseballGameBroadcastRadio = "'N/A'"
    return baseballGameBroadcastRadio

플레이한 구장

기본 값: 'N/A'

업데이트된 값: '잠실'

In [ ]:
def GetBaseballPlayedPlace(webCrawl):
    baseballGamePlayedPlace = "'N/A'"
    try:
        webCrawlBaseballGamePlayedPlaceClass = GetSpecialClassData("ballpark", webCrawl)
        baseballGamePlayedPlace = webCrawlBaseballGamePlayedPlaceClass.text
        baseballGamePlayedPlace = "'" + baseballGamePlayedPlace + "'"
    except:
        baseballGamePlayedPlace = "'N/A'"
    return baseballGamePlayedPlace

비고

기본 값: 'N/A'

업데이트된 값: '-'

In [ ]:
def GetBaseballPlayedEtc(webCrawl):
    baseballGamePlayedEtc = "'N/A'"
    try:
        webCrawlBaseballGamePlayedEtcClass = GetSpecialClassData("etc", webCrawl)
        baseballGamePlayedEtc = webCrawlBaseballGamePlayedEtcClass.text
        baseballGamePlayedEtc = "'" + baseballGamePlayedEtc + "'"
    except:
        baseballGamePlayedEtc = "'N/A'"
    return baseballGamePlayedEtc

# 크롤링을 위한 실제 코딩 파트

크롤링할 타겟 링크를 설정 ex) kbo정규 시즌 경기 결과표

In [ ]:
ConnectToTargetUrl("http://www.koreabaseball.com/Schedule/GameList/General.aspx")

In [ ]:
print GetNowSearchedDate(webCrawl)

# 프로세싱 규칙
## 한번 쭉 긁어온 다음 다 긁었으면 전 년/월 조회 클릭

In [ ]:
def CrawlDataFromTargetPage(webCrawl):
    scheduleData = GetSearchedScheduleBoardList(webCrawl)
    nowSearchingDate = GetNowSearchedDate(webCrawl)

    baseballPlayedDate = "00"
    baseballPlayedTime = "00:00"
    baseballPlayedTeams = ""
    baseballPlayedScore = ""
    baseballGameDetailLink = ""
    baseballBroadcastTV = ""
    baseballBroadcastRadio = ""
    baseballPlayedPlace = ""
    baseballPlayedEtc = ""
    crawledDate = ""
    tempToCompare = ""
    sqlQuery = ""

    for index in scheduleData:
        #print index
        print ""
    
        tempToCompare = GetBaseballPlayedDate(index)
        if tempToCompare != "00":
            baseballPlayedDate = tempToCompare
        tempToCompare = GetBaseballPlayedTime(index)
        if tempToCompare != "00:00":
            baseballPlayedTime = tempToCompare
        else:
            baseballPlayedTime = "00:00"
        #print nowSearchingDate + "-" + baseballPlayedDate + " " + baseballPlayedTime + ":00"
        baseballPlayedTeams = GetBaseballPlayedTeamsName(index)
        baseballPlayedScore = GetBaseballPlayedScore(index)
        baseballGameDetailLink = GetBaseballGameDetailLink(index)
        baseballBroadcastTV = GetBaseballBroadcastTV(index)
        baseballBroadcastRadio = GetBaseballBroadcastRadio(index)
        baseballPlayedPlace = GetBaseballPlayedPlace(index)
        baseballPlayedEtc = GetBaseballPlayedEtc(index)
        crawledDate = nowSearchingDate + "-" + baseballPlayedDate + " " + baseballPlayedTime + ":00"
        sqlQuery = MakeQueryWithParameter(crawledDate,
                                    baseballPlayedTeams, baseballPlayedScore, baseballGameDetailLink, 
                                    baseballBroadcastTV, baseballBroadcastRadio, baseballPlayedPlace, baseballPlayedEtc)
        print sqlQuery
        InsertDataToDB(sqlQuery)
        #break
    #return crawledDate

In [ ]:
while(1):
    try:
        CrawlDataFromTargetPage(webCrawl)
        try:
            GetPreviewsBaseballGameDataClick(webCrawl)
        except:
            print "It Seems Process Is Done"
            break;
    except:
        print "Exception Noticed"